In [2]:
import pandas as pd
import psycopg2
import datetime
import pytz

## What is the purpose of this script:

I want to know:
* which datasets recently published (2018) have a lot of incomplete records (here I set the threshold to 100% incomplete records),
* and what is the most frequent reason for incompleteness.

Then I want to aggregate this information per endorsing node and their continent.


## First, get the information from hive

The definition on incompleteness can be found on any [country page](https://www.gbif.org/country/SE/publishing), it is the following:
>A record is here defined to be complete if it includes an identification at least to species rank, valid coordinates, a full date of occurrence and a given basis of record (e.g. Observation, specimen etc).

This is the query I ran, it is based on the [following ones](https://github.com/gbif/analytics/blob/master/hive/process/occ_complete_v2.q) (which are used to calculate the number of complete records accross snapshots).

I decided not to use the snapshots.

This query gives us mor infomration than needed but I thought it would be usedful to have, in case we get asked more details about the record completeness.

Unlike for the [queries this was inspired from](https://github.com/gbif/analytics/blob/master/hive/process/occ_complete_v2.q), this is grouped by datasets instead of by snapshots.

``` sql

CREATE EXTERNAL TABLE mgrosjean.datasets_completeness_query20210126 (
  datasetkey STRING,
  basisofrecord STRING,
  rank STRING,
  geospatial STRING,
  temporal STRING,
  count INT
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE LOCATION '/user/mgrosjean/datasets_completeness_query20210126.csv'


INSERT OVERWRITE TABLE mgrosjean.datasets_completeness_query20210126
SELECT
datasetkey,
basisofrecord,
COALESCE(
    CASE WHEN species IS NOT NULL AND species != taxonid THEN 'Infraspecies' ELSE NULL END,
    CASE WHEN species IS NOT NULL THEN 'Species' ELSE NULL END,
    CASE WHEN genus IS NOT NULL THEN 'Genus' ELSE NULL END,
    CASE WHEN family IS NOT NULL THEN 'Family' ELSE NULL END,
    CASE WHEN order_ IS NOT NULL THEN 'Order' ELSE NULL END,
    CASE WHEN class IS NOT NULL THEN 'Class' ELSE NULL END,
    CASE WHEN phylum IS NOT NULL THEN 'Phylum' ELSE NULL END,
    CASE WHEN kingdom IS NOT NULL AND kingdom != 0 THEN 'Kingdom' ELSE NULL END,
    "Unknown"
  ) AS rank,
  COALESCE(
    CASE WHEN hascoordinate = TRUE THEN 'Georeferenced' ELSE NULL END,
    CASE WHEN countrycode IS NOT NULL THEN 'CountryOnly' ELSE NULL END,
    "Unknown"
  ) AS geospatial,
  COALESCE(
    CASE WHEN year IS NOT NULL AND month IS NOT NULL AND day IS NOT NULL THEN 'YearMonthDay' ELSE NULL END,
    CASE WHEN year IS NOT NULL AND month IS NOT NULL THEN 'YearMonth' ELSE NULL END,
    CASE WHEN year IS NOT NULL THEN 'YearOnly' ELSE NULL END,
    "Unknown"
  ) AS temporal,
  COUNT(*) AS count
FROM prod_h.occurrence
-- WHERE to_date(from_unixtime(cast(fragmentcreated/1000 as INT))) >= "2018-01-01" AND to_date(from_unixtime(cast(fragmentcreated/1000 as INT))) < "2019-01-01"
GROUP BY
  datasetkey,
  basisofrecord,
  COALESCE(
    CASE WHEN species IS NOT NULL AND species != taxonid THEN 'Infraspecies' ELSE NULL END,
    CASE WHEN species IS NOT NULL THEN 'Species' ELSE NULL END,
    CASE WHEN genus IS NOT NULL THEN 'Genus' ELSE NULL END,
    CASE WHEN family IS NOT NULL THEN 'Family' ELSE NULL END,
    CASE WHEN order_ IS NOT NULL THEN 'Order' ELSE NULL END,
    CASE WHEN class IS NOT NULL THEN 'Class' ELSE NULL END,
    CASE WHEN phylum IS NOT NULL THEN 'Phylum' ELSE NULL END,
    CASE WHEN kingdom IS NOT NULL AND kingdom != 0 THEN 'Kingdom' ELSE NULL END,
    "Unknown"
  ),
  COALESCE(
    CASE WHEN hascoordinate = TRUE THEN 'Georeferenced' ELSE NULL END,
    CASE WHEN countrycode IS NOT NULL THEN 'CountryOnly' ELSE NULL END,
    "Unknown"
  ),
  COALESCE(
    CASE WHEN year IS NOT NULL AND month IS NOT NULL AND day IS NOT NULL THEN 'YearMonthDay' ELSE NULL END,
    CASE WHEN year IS NOT NULL AND month IS NOT NULL THEN 'YearMonth' ELSE NULL END,
    CASE WHEN year IS NOT NULL THEN 'YearOnly' ELSE NULL END,
    "Unknown"
  );```
  
## Second, get information from the registry

This is to be able to summarise the completness information per year of dataset created and per node/continent.

In [3]:
conn = psycopg2.connect(dbname='prod_b_registry', 
                        user='', 
                        host='pg1.gbif.org', 
                        password='')
cur = conn.cursor()
# This is the query for the registry
query = """SELECT d.key, d.created, node.title, node.continent, node.country
           FROM dataset d JOIN organization o ON d.publishing_organization_key = o.key
           JOIN node ON o.endorsing_node_key = node.key;"""
# Queries the GBIF registry
cur.execute(query)
res_query = cur.fetchall()

dataset_per_publishing_country = pd.DataFrame(res_query, columns=['UUID',
                                                                  'created',
                                                                  'nodeTitle',
                                                                  'nodeContinent',
                                                                  'nodeCountry'])
dataset_per_publishing_country = dataset_per_publishing_country.set_index("UUID")
cur.close()
conn.close()

## Load information from saved hive table and process it

In [4]:
dq_all_path = "../datasets_completeness_query20210126.csv"
dq_all = pd.read_table(dq_all_path, sep = "\t", names = ["datasetkey",
                                                         "basisofrecord",
                                                         "rank",
                                                         "geospatial",
                                                         "temporal",
                                                         "count"])

In [5]:
def complete_name(dq):
    return ((dq["rank"] == "Species") | (dq["rank"] == "Infraspecies"))

def complete_basisofrecord(dq):
    return dq["basisofrecord"] != "UNKNOWN"
    
def complete_geospatial(dq):
    return dq["geospatial"] == "Georeferenced"

def complete_temporal(dq):
    return dq["temporal"] == "YearMonthDay"

def complete_records(dq_year):
    '''
    return a mask for a given dataset of complete records
    '''
    complete = complete_name(dq_year) & complete_basisofrecord(dq_year) & complete_geospatial(dq_year) & complete_temporal(dq_year)
    return complete

In [6]:
def number_of_complete_records_for_year(dq_year):
    '''
    Return number of complete and total records published in a given year
    in addition to the main issues for incomplete records
    '''
    records_stats = pd.DataFrame(columns=["complete", "total"],
                                 index=dq_year.groupby(["datasetkey"]).first().index)
    complete = complete_records(dq_year)
    records_stats["complete"] = dq_year[complete].groupby(["datasetkey"])["count"].sum()
    records_stats["total"] = dq_year.groupby(["datasetkey"])["count"].sum()
    records_stats = records_stats.fillna(0)
    
    # Check which is the most frequent incompleteness reason
    reason_incomplete = dq_year[~complete].sort_values(["datasetkey", "count"],
                                                       ascending=False).groupby("datasetkey").first()
    records_stats = pd.concat([records_stats, reason_incomplete], axis = 1)
    return records_stats

In [7]:
def select_ds_by_year_registry(dataset_per_publishing_country, year):
    '''
    return a mask to select datasets created a certain year
    '''
    before_that_year = pd.to_datetime(dataset_per_publishing_country.created, utc=True) >= datetime.datetime(year=year, month=1, day=1, tzinfo = pytz.UTC)
    after_that_year = pd.to_datetime(dataset_per_publishing_country.created, utc=True) < datetime.datetime(year=year+1, month=1, day=1, tzinfo = pytz.UTC)
    return (before_that_year & after_that_year)

In [8]:
def completeness_for_ds_published_in_year(dq_year, dataset_per_publishing_country, year):
    '''
    returns datasets filtered for a given year and merged
    '''
    dq_stats = number_of_complete_records_for_year(dq_year)
    mask = select_ds_by_year_registry(dataset_per_publishing_country, year)
    dq_ds_stats = pd.concat([dataset_per_publishing_country[mask], dq_stats], axis = 1, join = 'inner')

    return dq_ds_stats

In [9]:
def main_incompleteness_reason(dataset):
    '''
    For a given subset, return the most frequent reason
    for data incompleteness
    '''
    res = pd.DataFrame(columns=["freq"])
    res.at["basis of record", "freq"] = dataset[~complete_basisofrecord(dataset)].shape[0]
    res.at["taxon rank", "freq"] = dataset[~complete_name(dataset)].shape[0]
    res.at["geospatial", "freq"] = dataset[~complete_geospatial(dataset)].shape[0]
    res.at["temporal", "freq"] = dataset[~complete_temporal(dataset)].shape[0]
    return res.idxmax()[0]

In [10]:
def groupby_stats(dq_year, dataset_per_publishing_country, groupby, threshold, year):
    '''
    Aggregates statistics per node or continent
    '''
    # Column titles
    percentage_below_thresh = "Pct of datasets registered in "+str(year)+" with incomplete data"
    incompleteness = "Main reason of incompleteness right now"
    
    # Calculate the number of complete records per dataset
    datasets_stats = completeness_for_ds_published_in_year(dq_year, dataset_per_publishing_country, year)
    datasets_stats["complete_datasets_mask"] = datasets_stats["complete"]/datasets_stats["total"] <= threshold
    
    # Aggregate statistics
    stats_per_groupby = pd.DataFrame(columns=[percentage_below_thresh])
    # Calculate percentage of datasets with incomplete data records
    stats_per_groupby[percentage_below_thresh] = datasets_stats.groupby(groupby)["complete_datasets_mask"].sum()*100/datasets_stats.groupby([groupby])["complete_datasets_mask"].count()
    # Get most frequent reason for incompleteness
    for subgroup in stats_per_groupby.index.tolist():
        subset = datasets_stats[datasets_stats[groupby] == subgroup]
        stats_per_groupby.at[subgroup, incompleteness] = main_incompleteness_reason(subset)
    return stats_per_groupby

In [11]:
stats_per_continent_2020 = groupby_stats(dq_all, dataset_per_publishing_country, "nodeContinent", 0, 2020)
stats_per_continent_2019 = groupby_stats(dq_all, dataset_per_publishing_country, "nodeContinent", 0, 2019)
stats_per_continent_2018 = groupby_stats(dq_all, dataset_per_publishing_country, "nodeContinent", 0, 2018)
stats_per_continent_2017 = groupby_stats(dq_all, dataset_per_publishing_country, "nodeContinent", 0, 2017)
stats_per_node_2020 = groupby_stats(dq_all, dataset_per_publishing_country, "nodeTitle", 0, 2020)
stats_per_node_2019 = groupby_stats(dq_all, dataset_per_publishing_country, "nodeTitle", 0, 2019)
stats_per_node_2018 = groupby_stats(dq_all, dataset_per_publishing_country, "nodeTitle", 0, 2018)
stats_per_node_2017 = groupby_stats(dq_all, dataset_per_publishing_country, "nodeTitle", 0, 2017)

/Users/gjq527/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [12]:
stats_per_continent = pd.concat([stats_per_continent_2020, stats_per_continent_2019, stats_per_continent_2018, stats_per_continent_2017], axis=1)
stats_per_node = pd.concat([stats_per_node_2020, stats_per_node_2019, stats_per_node_2018, stats_per_node_2017], axis=1)

/Users/gjq527/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [13]:
writer = pd.ExcelWriter('data_quality_per_dataset_for_2020.xlsx')
stats_per_continent.to_excel(writer,'CONTINENT')
stats_per_node.to_excel(writer,'ENDOSING NODE')
writer.save()

In [15]:
stats_per_node

,Pct of datasets registered in 2020 with incomplete data,Main reason of incompleteness right now,Pct of datasets registered in 2019 with incomplete data,Main reason of incompleteness right now,Pct of datasets registered in 2018 with incomplete data,Main reason of incompleteness right now,Pct of datasets registered in 2017 with incomplete data,Main reason of incompleteness right now
Albertine Rift Conservation Society,33.333333,taxon rank,0.000000,taxon rank,50.000000,taxon rank,NaN,NaN
Asean Centre for Biodiversity,100.000000,temporal,100.000000,geospatial,NaN,NaN,100.000000,geospatial
Atlas of Living Australia,8.695652,taxon rank,18.604651,taxon rank,29.113924,taxon rank,31.736527,taxon rank
Belgian Biodiversity Platform,84.090909,basis of record,55.172414,temporal,23.076923,taxon rank,22.222222,taxon rank
BioCASE,0.000000,temporal,0.000000,geospatial,NaN,NaN,0.000000,basis of record
...,...,...,...,...,...,...,...,...
South African Biodiversity Information Facility,68.421053,taxon rank,52.941176,taxon rank,66.666667,temporal,42.857143,taxon rank
Taiwan Biodiversity Information Facility,22.222222,taxon rank,22.222222,taxon rank,50.000000,taxon rank,33.333333,taxon rank
Tanzania Biodiversity Information Facility,0.000000,taxon rank,100.000000,temporal,50.000000,temporal,NaN,NaN
U.S. Geological Survey,30.890052,taxon rank,13.043478,geospatial,28.070175,geospatial,25.333333,geospatial
